In [12]:
import sys
sys.path.append('/Users/jsh/code/drivendata/spacecrafts/')
import shutil
from pathlib import Path
import pandas as pd
import torch
import torchvision
import pytorch_lightning as L
from submission_src.spacecraft_datamodule import SpacecraftDataModule, SpacecraftDataModule, collate_fn
from submission_src.spacecraft_model import SpacecraftModel
from submission_src.MyFasterRCNN import MyFasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.ssd import SSDClassificationHead
from torchvision.models.detection._utils import retrieve_out_channels

In [13]:
# Set configuration variables
PROJ_DIRECTORY = Path.cwd().parent
DATA_DIRECTORY = PROJ_DIRECTORY / "data"
DEV_DIRECTORY = PROJ_DIRECTORY / "data_dev"
IMAGES_DIRECTORY = DEV_DIRECTORY / "images"
SUBMISSION_SRC_DIR = PROJ_DIRECTORY / "submission_src"
IMAGE_SIZE = (1280, 1024)
IMAGE_RESIZE_SCALE = (1/8)
IMAGE_RESIZED = (int(IMAGE_SIZE[0]*IMAGE_RESIZE_SCALE), int(IMAGE_SIZE[1]*IMAGE_RESIZE_SCALE))
VALIDATION_IDS_PATH = PROJ_DIRECTORY / 'notebooks' / 'validation_ids.csv'

In [14]:
# Import img ids for validation set used on Kaggle.
validation_ids = pd.read_csv(VALIDATION_IDS_PATH)['0'].tolist()

# Move validation set images to data/images directory.
dest_dir = DATA_DIRECTORY / "images"
if not dest_dir.exists():
    dest_dir.mkdir(parents=True, exist_ok=True)

for img in validation_ids:
    shutil.copy2(IMAGES_DIRECTORY / f"{img}.png", dest_dir / f"{img}.png")

# Create submission_format file that corresponds to the validation set.
submission_format = pd.read_csv(DEV_DIRECTORY / "submission_format.csv", index_col="image_id")
submission_format_val = submission_format.loc[validation_ids]
submission_format_val.to_csv(DATA_DIRECTORY / "submission_format.csv")

# Create train_labels file that corresponds to the validation set.
train_labels = pd.read_csv(DEV_DIRECTORY / "train_labels.csv", index_col="image_id")
test_labels = train_labels.loc[validation_ids]
test_labels.to_csv(DATA_DIRECTORY / "test_labels.csv")

In [ ]:
# Download base model.
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(weights="DEFAULT")

# Adjust the model for spacecraft recognition task.
num_classes = 2 # 2 classes are spacecraft or background
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace pre-trained head with new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)



In [31]:
def get_ssd():  
    # Download base model
    weights = torchvision.models.detection.SSDLite320_MobileNet_V3_Large_Weights.DEFAULT
    model = torchvision.models.detection.ssdlite320_mobilenet_v3_large(weights=weights)

    num_classes = 2 # 2 classes are spacecraft or background
    num_anchors = model.anchor_generator.num_anchors_per_location()
    in_channels = retrieve_out_channels(model.backbone, (320,320))

    # Replace head for Spacecraft detection task
    model.head.classification_head = SSDClassificationHead(in_channels=in_channels, num_anchors=num_anchors, num_classes=num_classes)

    model.train()
    return model

def get_faster_rcnn():
    # Download base model
    weights = torchvision.models.detection.FasterRCNN_MobileNet_V3_Large_320_FPN_Weights.DEFAULT
    model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(weights=weights)

    # Adjust the model for spacecraft recognition task.
    num_classes = 2 # 2 classes are spacecraft or background
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace pre-trained head with new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model.train()
    return model

In [22]:
!ls ../data/images/ | wc -l

    4801


In [29]:
ckpt_path = '/Users/jsh/code/drivendata/spacecrafts/notebooks/lightning_logs/version_5/checkpoints/epoch=9-step=6570.ckpt'
checkpoint = torch.load(ckpt_path, map_location=torch.device('cpu'))
state_dict = checkpoint['state_dict']

In [23]:
state_dict = torch.load('ssd_state_dict.pt', map_location=torch.device('cpu'))

In [24]:
model = get_ssd()

Downloading: "https://download.pytorch.org/models/ssdlite320_mobilenet_v3_large_coco-a79551df.pth" to /Users/jsh/.cache/torch/hub/checkpoints/ssdlite320_mobilenet_v3_large_coco-a79551df.pth
100%|██████████| 13.4M/13.4M [00:01<00:00, 9.17MB/s]


In [36]:
# Load model state_dict and save full version to the submission_src folder.
model = get_faster_rcnn()
frcnn = MyFasterRCNN(model)
frcnn.load_state_dict(state_dict)

<All keys matched successfully>

In [37]:
frcnn.learning_rate

0.001

In [25]:
# Load model state_dict and save full version to the submission_src folder.
ssd = SpacecraftModel(model)
ssd.load_state_dict(state_dict)
torch.save(model, SUBMISSION_SRC_DIR / 'ssd_1epoch.pt')

In [26]:
!cd {PROJ_DIRECTORY} && make pack-submission

mkdir -p submission/
cd submission_src; zip -r ../submission/submission.zip ./*
  adding: MyFasterRCNN.py (deflated 63%)
  adding: __pycache__/ (stored 0%)
  adding: __pycache__/spacecraft_model.cpython-310.pyc (deflated 50%)
  adding: __pycache__/MyFasterRCNN.cpython-310.pyc (deflated 49%)
  adding: __pycache__/spacecraft_datamodule.cpython-310.pyc (deflated 50%)
  adding: frcnn10epochs.pt (deflated 7%)
  adding: lightning5.pt (deflated 7%)
  adding: main.py (deflated 62%)
  adding: main.sh (deflated 26%)
  adding: spacecraft_datamodule.py (deflated 67%)
  adding: spacecraft_model.py (deflated 64%)
  adding: spacecraft_utils.py (stored 0%)
  adding: ssd_1epoch.pt (deflated 8%)


In [27]:
! cd {PROJ_DIRECTORY} && make test-submission

mkdir -p submission/
chmod -R 0777 submission/
docker run \
		-it \
		--network none \
		--mount type=bind,source="/Users/jsh/code/drivendata/spacecrafts"/data,target=/code_execution/data,readonly \
		--mount type=bind,source="/Users/jsh/code/drivendata/spacecrafts"/submission,target=/code_execution/submission \
		--shm-size 8g \
		--name spacecraft-pose-object-detection \
		--rm \
		bc1d6eb59ba6
+ main
+ tee /code_execution/submission/log.txt
+ cd /code_execution
+ curl --silent --connect-timeout 10 --max-time 12 www.example.com
+ data_directory=/code_execution/data
+ format_filename=/code_execution/data/submission_format.csv
+ (( i=0 ))
+ (( i<=5 ))
+ t=0
+ '[' -f /code_execution/data/submission_format.csv ']'
+ echo 'found /code_execution/data/submission_format.csv after 0 seconds; data is mounted'
found /code_execution/data/submission_format.csv after 0 seconds; data is mounted
+ break
+ '[' '!' -f /code_execution/data/submission_format.csv ']'
+ expected_filename=main.sh
++ zip -s